Fichier de test des codes du projet

In [8]:
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Essayer d'importer les modules nécessaires
try:
    import yaml
except ImportError:
    install_package('pyyaml')
    import yaml

try:
    import pandas as pd
except ImportError:
    install_package('pandas')
    import pandas as pd

try:
    import geopandas as gpd
except ImportError:
    install_package('geopandas')
    import geopandas as gpd

try:
    import plotly
except ImportError:
    install_package('plotly')
    import plotly

try:
    import plotnine
except ImportError:
    install_package('plotnine')
    import plotnine

In [9]:
#Exercice 1
import yaml
def create_data_list(source_file):
    data = yaml.safe_load(source_file)
    return data

print(create_data_list("sources.yml"))

sources.yml


In [10]:
print("aa")

aa


Exercice 2

In [11]:
from src.create_data_list import create_data_list
import pandas as pd
urls = create_data_list("sources.yml")
list_urls = (list(urls['airports'].values()))

def clean_dataframe(df):
    # Create 'an' and 'mois' columns
    df['an'] = df['ANMOIS'].astype(str).str[:4]
    df['mois'] = df['ANMOIS'].astype(str).str[-2:]

    # Remove leading zeros from 'mois' column
    df['mois'] = df['mois'].str.replace(r'^0+', '', regex=True)

    # Convert all column names to lowercase
    df.columns = df.columns.str.lower()

    return df

def import_airport_data(list_files):
    # Define the data types for each column
    col_types = {
        "ANMOIS": "str",
        "APT": "str",     # equivalent to col_character()
        "APT_NOM": "str", # equivalent to col_character()
        "APT_ZON": "str", # equivalent to col_character()
    }

    # Read the CSV file(s) with the specified column types
    pax_apt_all = pd.concat([
        pd.read_csv(file, delimiter = ';', dtype = col_types)
        for file in list_files
        ])

    # Clean the DataFrame (assuming clean_dataframe is a predefined function)
    pax_apt_all = clean_dataframe(pax_apt_all)

    return pax_apt_all


def import_compagnies_data(list_files):
    # Define the data types for each column
    col_types = {
        "ANMOIS": "str",
        "CIE": "str",
        "CIE_NOM": "str",
        "CIE_NAT": "str",
        "CIE_PAYS": "str"
    }

    # Read the CSV file(s) with the specified column types
    pax_cie_all = pd.concat([
        pd.read_csv(file, delimiter = ';', dtype = col_types)
        for file in list_files
        ])

    # Clean the DataFrame (assuming clean_dataframe is a predefined function)
    pax_cie_all = clean_dataframe(pax_cie_all)


    return pax_cie_all


def import_liaisons_data(list_files):
    # Define the data types for each column
    col_types = {
        "ANMOIS": "str",
        "LSN": "str",
        "LSN_DEP_NOM": "str",
        "LSN_ARR_NOM": "str",
        "LSN_SCT": "str",
        "LSN_FSC": "str"
    }

    # Read the CSV file(s) with the specified column types
    pax_lsn_all = pd.concat([
        pd.read_csv(file, delimiter = ';', dtype = col_types)
        for file in list_files
        ])

    # Clean the DataFrame
    pax_lsn_all = clean_dataframe(pax_lsn_all)

    return pax_lsn_all

Partie 4, script main

In [12]:
#Main
import pandas as pd
import geopandas as gpd
import plotly.express as px
from plotnine import ggplot, geom_line, aes



# Load data ----------------------------------
urls = create_data_list("./sources.yml")


pax_apt_all = import_airport_data(urls['airports'].values())
pax_cie_all = import_airport_data(urls['compagnies'].values())
pax_lsn_all = import_airport_data(urls['liaisons'].values())


airports_location = gpd.read_file(
    urls['geojson']['airport']
)


liste_aeroports = pax_apt_all['apt'].unique()
default_airport = liste_aeroports[0]

In [13]:
pax_apt_all['trafic'] = pax_apt_all['apt_pax_dep'] + \
  pax_apt_all['apt_pax_tr'] + \
  pax_apt_all['apt_pax_arr']
trafic_aeroports = (
  pax_apt_all
  .loc[pax_apt_all['apt'] == default_airport]
)
trafic_aeroports['date'] = pd.to_datetime(
  trafic_aeroports['anmois'] + '01', format='%Y%m%d'
)

/tmp/ipykernel_27869/574079793.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
trafic_aeroports.head(5)

,anmois,apt,apt_nom,apt_zon,apt_peq,apt_pax_dep,apt_pax_arr,apt_pax_tr,apt_frp_dep,apt_frp_arr,apt_nmvt_mxt,apt_nmvt_cgo,an,mois,trafic,date
0,201801,FMCZ,MAYOTTE-MARCEL HENRY,OM,35209,13108,18977,1202,"21,8","170,4",546,1,2018,1,33287,2018-01-01
88,201802,FMCZ,MAYOTTE-MARCEL HENRY,OM,26844,14135,9642,937,31,182,382,0,2018,2,24714,2018-02-01
174,201803,FMCZ,MAYOTTE-MARCEL HENRY,OM,29207,12863,13169,929,"25,3","199,4",413,1,2018,3,26961,2018-03-01
262,201804,FMCZ,MAYOTTE-MARCEL HENRY,OM,28441,12778,11870,1576,"49,1","172,6",381,0,2018,4,26224,2018-04-01
352,201805,FMCZ,MAYOTTE-MARCEL HENRY,OM,32873,14675,14185,1572,"39,1",205,442,0,2018,5,30432,2018-05-01
